In [81]:
import numpy as np
import pandas as pd
import os
import glob
import mne

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

In [45]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

# Analysis Plan
## Experiments 
1. Left vs. Right
2. Inward vs. Upward vs. Downward
3. Open and close vs. Rest
4. Open vs. Close
    1. ROIs
    2. Contralteral and Ipsilateral classification
    3. Non-ROI classification

## Features
- [x] Power Spectral Density
- [x] Spectopo Plots
- [x] Time-series
- [x] ICA Activations
- [ ] ERS and ERD

## Algorithms
1. ML
    1. PSD [band-wise and entire-band]
    2. Time-series
2. CNN
    1. spectopo
3. LSTM
    1. time-series


# 1. Time-domain classification

In [2]:
dataset = pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/cleaned_data_asr.csv")
dataset

,0,1,2,3,4,5,6,7,8,9,...,1096,1097,1098,1099,handPos,Hand,Action,Subject,Channel,Trial
0,-4.671667,-4.451136,-4.263363,-4.122293,-4.039534,-4.023682,-4.079801,-4.209121,-4.408905,-4.672575,...,-8.788404,-8.901701,-8.911717,-8.817698,palmDown,Right,Open,P01,1,1
1,6.391617,6.335880,6.244392,6.109434,5.925195,5.688357,5.398465,5.058200,4.673382,4.252845,...,-20.972265,-20.743320,-20.478785,-20.176245,palmDown,Right,Open,P01,2,1
2,0.160325,0.081283,-0.010762,-0.123249,-0.262893,-0.435155,-0.643720,-0.890082,-1.173304,-1.489870,...,7.087850,7.448699,7.722362,7.906469,palmDown,Right,Open,P01,3,1
3,-1.336037,-1.659968,-1.983633,-2.302723,-2.612850,-2.909530,-3.188195,-3.444230,-3.673032,-3.870121,...,10.234831,10.642152,11.007372,11.328000,palmDown,Right,Open,P01,4,1
4,12.797042,12.577418,12.352886,12.123366,11.889187,11.651197,11.410817,11.170057,10.931464,10.698002,...,4.104468,3.970622,3.816026,3.642702,palmDown,Right,Open,P01,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384955,-1.525676,-1.255532,-0.849529,-0.324107,0.302524,1.010990,1.781305,2.593337,3.427199,4.263574,...,-9.633747,-10.102177,-10.289121,-10.183593,palmUp,Right,Close,FP12,58,480
384956,0.430836,1.836212,3.301004,4.780484,6.231448,7.613512,8.890228,10.029982,11.006654,11.800056,...,-18.231979,-18.785727,-18.982012,-18.810005,palmUp,Right,Close,FP12,59,480
384957,2.463603,3.943370,5.494955,7.070804,8.621435,10.097134,11.449710,12.634233,13.610693,14.345462,...,-9.042601,-8.470536,-7.647579,-6.599504,palmUp,Right,Close,FP12,60,480
384958,-4.482436,-4.953345,-5.120734,-4.995581,-4.599613,-3.963956,-3.127460,-2.134715,-1.033886,0.125495,...,-4.829801,-5.120802,-5.356338,-5.527806,palmUp,Right,Close,FP12,61,480


In [42]:
chans, chan_count = np.unique(dataset["Channel"], return_counts=True)
chans, chan_count

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 array([5760, 6240, 6240, 6240, 6240, 6240, 5760, 6240, 6240, 5760, 6240,
        6240, 6240, 6240, 6240, 6240, 6240, 6240, 6240, 5760, 5760, 6240,
        6240, 5760, 6240, 6240, 6240, 6240, 6240, 6240, 6240, 6240, 6240,
        6240, 4800, 6240, 5280, 6240, 4800, 6240, 5280, 6240, 6240, 6240,
        6240, 6240, 6240, 6240, 5760, 6240, 5760, 5280, 6240, 6240, 6240,
        6240, 6240, 6240, 6240, 6240, 6240, 4320, 3840, 5760]))

In [43]:
# Find list of channels available for every participant
good_chans = [i+1 for i in range(len(chan_count)) if chan_count[i] == 6240]
print(len(good_chans))
print(good_chans)

48
[2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 55, 56, 57, 58, 59, 60, 61]


In [44]:
# Subject-wise channel availability
sub_good_chans = {}
for sub in np.unique(dataset["Subject"]):
    sub_good_chans[sub] = np.unique(dataset[dataset["Subject"] == sub]["Channel"]) 
sub_good_chans

{'FP02': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP03': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP05': array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP06': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 26

In [76]:
def create_dataset():
    X, y_lr, y_pos, y_act = [], [], [], []
    for trial in range(1, 480):
        print("Current Trial: ", trial)
        for sub in np.unique(dataset.Subject):
            for pos in ["palmDown", "palmUp", "palmIn"]:
                for action in ["Open", "Close"]:
                    for hand in ["Left", "Right"]:
                        trial_eeg = dataset[
                            (dataset["Subject"] == sub) & 
                            (dataset["handPos"] == pos) &
                            (dataset["Trial"] == trial) &
                            (dataset["Action"] == action) &
                            (dataset["Channel"].isin(good_chans)) &
                            (dataset["Hand"] == hand) 
                        ]
                        if trial_eeg.shape[0] == 0:
                            continue
                        X.append(trial_eeg.iloc[:, :-6])
                        y_lr.append(trial_eeg.iloc[0, -5])  
                        y_pos.append(trial_eeg.iloc[0, -6])  
                        y_act.append(trial_eeg.iloc[0, -4])
        
    return np.array(X), np.array(y_lr), np.array(y_pos), np.array(y_act)

X, y_lr, y_pos, y_act = create_dataset()

Current Trial:  1
Current Trial:  2
Current Trial:  3
Current Trial:  4
Current Trial:  5
Current Trial:  6
Current Trial:  7
Current Trial:  8
Current Trial:  9
Current Trial:  10
Current Trial:  11
Current Trial:  12
Current Trial:  13
Current Trial:  14
Current Trial:  15
Current Trial:  16
Current Trial:  17
Current Trial:  18
Current Trial:  19
Current Trial:  20
Current Trial:  21
Current Trial:  22
Current Trial:  23
Current Trial:  24
Current Trial:  25
Current Trial:  26
Current Trial:  27
Current Trial:  28
Current Trial:  29
Current Trial:  30
Current Trial:  31
Current Trial:  32
Current Trial:  33
Current Trial:  34
Current Trial:  35
Current Trial:  36
Current Trial:  37
Current Trial:  38
Current Trial:  39
Current Trial:  40
Current Trial:  41
Current Trial:  42
Current Trial:  43
Current Trial:  44
Current Trial:  45
Current Trial:  46
Current Trial:  47
Current Trial:  48
Current Trial:  49
Current Trial:  50
Current Trial:  51
Current Trial:  52
Current Trial:  53
Cu

Current Trial:  417
Current Trial:  418
Current Trial:  419
Current Trial:  420
Current Trial:  421
Current Trial:  422
Current Trial:  423
Current Trial:  424
Current Trial:  425
Current Trial:  426
Current Trial:  427
Current Trial:  428
Current Trial:  429
Current Trial:  430
Current Trial:  431
Current Trial:  432
Current Trial:  433
Current Trial:  434
Current Trial:  435
Current Trial:  436
Current Trial:  437
Current Trial:  438
Current Trial:  439
Current Trial:  440
Current Trial:  441
Current Trial:  442
Current Trial:  443
Current Trial:  444
Current Trial:  445
Current Trial:  446
Current Trial:  447
Current Trial:  448
Current Trial:  449
Current Trial:  450
Current Trial:  451
Current Trial:  452
Current Trial:  453
Current Trial:  454
Current Trial:  455
Current Trial:  456
Current Trial:  457
Current Trial:  458
Current Trial:  459
Current Trial:  460
Current Trial:  461
Current Trial:  462
Current Trial:  463
Current Trial:  464
Current Trial:  465
Current Trial:  466


In [79]:
X.shape

(6227, 48, 1100)

In [57]:
trial_eeg

,0,1,2,3,4,5,6,7,8,9,...,1096,1097,1098,1099,handPos,Hand,Action,Subject,Channel,Trial


In [91]:
X_red = X[:, :, :400]
X_red.shape

(6227, 48, 400)

In [94]:
X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])).shape

(6227, 19200)

## 1. Left vs. Right

In [95]:
X_train, X_eval, y_train, y_eval = train_test_split(
    X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])), y_lr, test_size=0.2, random_state=42
)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4981, 19200), (1246, 19200), (4981,), (1246,))

In [96]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [36:08<00:00, 74.77s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.89,0.89,None,0.89,137.23
LGBMClassifier,0.88,0.88,None,0.88,43.02
NuSVC,0.87,0.87,None,0.87,171.21
ExtraTreesClassifier,0.86,0.86,None,0.86,13.90
RandomForestClassifier,0.86,0.86,None,0.86,44.01
AdaBoostClassifier,0.84,0.84,None,0.84,461.71
Perceptron,0.83,0.83,None,0.83,5.34
BaggingClassifier,0.83,0.83,None,0.83,600.93
PassiveAggressiveClassifier,0.82,0.82,None,0.82,14.92


In [97]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/time-left-vs-right-400ms.csv")

## 2. Inward vs. upward vs. downward

In [98]:
X_train, X_eval, y_train, y_eval = train_test_split(
    X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])), y_pos, test_size=0.2, random_state=42
)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4981, 19200), (1246, 19200), (4981,), (1246,))

In [99]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|████████████████████████████████████████| 29/29 [1:26:57<00:00, 179.91s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.44,0.44,None,0.44,197.66
ExtraTreesClassifier,0.42,0.42,None,0.42,24.16
RandomForestClassifier,0.41,0.41,None,0.41,63.47
SVC,0.41,0.41,None,0.41,302.42
NuSVC,0.40,0.40,None,0.40,316.26
AdaBoostClassifier,0.38,0.38,None,0.38,476.71
GaussianNB,0.37,0.38,None,0.36,3.95
BaggingClassifier,0.37,0.37,None,0.37,698.39
LogisticRegression,0.36,0.36,None,0.36,10.21


In [100]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/time-in-vs-up-vs-down-400ms.csv")

## 3. Open vs. close

In [101]:
X_train, X_eval, y_train, y_eval = train_test_split(
    X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])), y_act, test_size=0.2, random_state=42
)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4981, 19200), (1246, 19200), (4981,), (1246,))

In [104]:
clf = LazyClassifier(verbose=0,ignore_warnings=False, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

  0%|                                                    | 0/29 [04:35<?, ?it/s]


KeyboardInterrupt: 

In [103]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/time-open-vs-close-400ms.csv")

## Open/Close vs. Rest

In [105]:
dataset = pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/cleaned_data_asr_fix_v1.csv")
dataset

,0,1,2,3,4,5,6,7,8,9,...,997,998,999,Event,handPos,Hand,Action,Subject,Channel,Trial
0,-6.92,-7.04,-7.18,-7.35,-7.52,-7.69,-7.85,-7.98,-8.07,-8.12,...,-9.20,-9.17,-9.16,Fix,palmDown,Right,Open,P01,1,0
1,-7.83,-7.64,-7.46,-7.27,-7.09,-6.90,-6.71,-6.52,-6.32,-6.12,...,-0.10,-0.29,-0.48,Fix,palmDown,Right,Open,P01,2,0
2,-7.87,-7.54,-7.17,-6.76,-6.32,-5.87,-5.41,-4.96,-4.53,-4.13,...,-2.77,-2.62,-2.52,Fix,palmDown,Right,Open,P01,3,0
3,-1.14,-0.93,-0.75,-0.61,-0.51,-0.45,-0.43,-0.44,-0.48,-0.56,...,-4.28,-3.89,-3.55,Fix,palmDown,Right,Open,P01,4,0
4,-8.30,-8.16,-7.96,-7.71,-7.40,-7.05,-6.65,-6.23,-5.78,-5.31,...,8.10,7.97,7.78,Fix,palmDown,Right,Open,P01,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769915,8.82,8.21,7.50,6.72,5.91,5.08,4.27,3.50,2.80,2.19,...,-10.10,-10.29,-10.18,Action,palmUp,Right,Close,FP12,58,959
769916,12.37,11.62,10.73,9.72,8.63,7.49,6.32,5.17,4.05,3.01,...,-18.79,-18.98,-18.81,Action,palmUp,Right,Close,FP12,59,959
769917,6.91,6.91,6.89,6.86,6.82,6.75,6.65,6.51,6.31,6.05,...,-8.47,-7.65,-6.60,Action,palmUp,Right,Close,FP12,60,959
769918,4.53,4.61,4.67,4.71,4.75,4.78,4.82,4.87,4.93,5.00,...,-5.12,-5.36,-5.53,Action,palmUp,Right,Close,FP12,61,959


In [111]:
chans, chan_count = np.unique(dataset["Channel"], return_counts=True)
chans, chan_count

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 array([11520, 12480, 12480, 12480, 12480, 12480, 11520, 12480, 12480,
        11520, 12480, 12480, 12480, 12480, 12480, 12480, 12480, 12480,
        12480, 11520, 11520, 12480, 12480, 11520, 12480, 12480, 12480,
        12480, 12480, 12480, 12480, 12480, 12480, 12480,  9600, 12480,
        10560, 12480,  9600, 12480, 10560, 12480, 12480, 12480, 12480,
        12480, 12480, 12480, 11520, 12480, 11520, 10560, 12480, 12480,
        12480, 12480, 12480, 12480, 12480, 12480, 12480,  8640,  7680,
        11520]))

In [112]:
# Find list of channels available for every participant
good_chans = [i+1 for i in range(len(chan_count)) if chan_count[i] == 12480]
print(len(good_chans))
print(good_chans)

48
[2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 55, 56, 57, 58, 59, 60, 61]


In [113]:
# Subject-wise channel availability
sub_good_chans = {}
for sub in np.unique(dataset["Subject"]):
    sub_good_chans[sub] = np.unique(dataset[dataset["Subject"] == sub]["Channel"]) 
sub_good_chans

{'FP02': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP03': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP05': array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP06': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 26

In [122]:
def create_dataset(dataset):
    X, y_lr, y_pos, y_act, y_event = [], [], [], [], []
    n_trials = len(np.unique(dataset.Trial))
    for trial in range(n_trials):
        print("Current Trial: ", trial)
        for sub in np.unique(dataset.Subject):
            for pos in ["palmDown", "palmUp", "palmIn"]:
                for action in ["Open", "Close"]:
                    for hand in ["Left", "Right"]:
                        trial_eeg = dataset[
                            (dataset["Subject"] == sub) & 
                            (dataset["handPos"] == pos) &
                            (dataset["Trial"] == trial) &
                            (dataset["Action"] == action) &
                            (dataset["Channel"].isin(good_chans)) &
                            (dataset["Hand"] == hand) 
                        ]
                        if trial_eeg.shape[0] == 0:
                            continue
                        X.append(trial_eeg.iloc[:, :-6])
                        y_act.append(trial_eeg.iloc[0, -4])
                        y_lr.append(trial_eeg.iloc[0, -5])  
                        y_pos.append(trial_eeg.iloc[0, -6])  
                        y_event.append(trial_eeg.iloc[0, -7])
        
    return np.array(X), np.array(y_lr), np.array(y_pos), np.array(y_act), np.array(y_event)

X, y_lr, y_pos, y_act, y_event = create_dataset(dataset)

Current Trial:  0
Current Trial:  1
Current Trial:  2
Current Trial:  3
Current Trial:  4
Current Trial:  5
Current Trial:  6
Current Trial:  7
Current Trial:  8
Current Trial:  9
Current Trial:  10
Current Trial:  11
Current Trial:  12
Current Trial:  13
Current Trial:  14
Current Trial:  15
Current Trial:  16
Current Trial:  17
Current Trial:  18
Current Trial:  19
Current Trial:  20
Current Trial:  21
Current Trial:  22
Current Trial:  23
Current Trial:  24
Current Trial:  25
Current Trial:  26
Current Trial:  27
Current Trial:  28
Current Trial:  29
Current Trial:  30
Current Trial:  31
Current Trial:  32
Current Trial:  33
Current Trial:  34
Current Trial:  35
Current Trial:  36
Current Trial:  37
Current Trial:  38
Current Trial:  39
Current Trial:  40
Current Trial:  41
Current Trial:  42
Current Trial:  43
Current Trial:  44
Current Trial:  45
Current Trial:  46
Current Trial:  47
Current Trial:  48
Current Trial:  49
Current Trial:  50
Current Trial:  51
Current Trial:  52
Cur

Current Trial:  416
Current Trial:  417
Current Trial:  418
Current Trial:  419
Current Trial:  420
Current Trial:  421
Current Trial:  422
Current Trial:  423
Current Trial:  424
Current Trial:  425
Current Trial:  426
Current Trial:  427
Current Trial:  428
Current Trial:  429
Current Trial:  430
Current Trial:  431
Current Trial:  432
Current Trial:  433
Current Trial:  434
Current Trial:  435
Current Trial:  436
Current Trial:  437
Current Trial:  438
Current Trial:  439
Current Trial:  440
Current Trial:  441
Current Trial:  442
Current Trial:  443
Current Trial:  444
Current Trial:  445
Current Trial:  446
Current Trial:  447
Current Trial:  448
Current Trial:  449
Current Trial:  450
Current Trial:  451
Current Trial:  452
Current Trial:  453
Current Trial:  454
Current Trial:  455
Current Trial:  456
Current Trial:  457
Current Trial:  458
Current Trial:  459
Current Trial:  460
Current Trial:  461
Current Trial:  462
Current Trial:  463
Current Trial:  464
Current Trial:  465


Current Trial:  826
Current Trial:  827
Current Trial:  828
Current Trial:  829
Current Trial:  830
Current Trial:  831
Current Trial:  832
Current Trial:  833
Current Trial:  834
Current Trial:  835
Current Trial:  836
Current Trial:  837
Current Trial:  838
Current Trial:  839
Current Trial:  840
Current Trial:  841
Current Trial:  842
Current Trial:  843
Current Trial:  844
Current Trial:  845
Current Trial:  846
Current Trial:  847
Current Trial:  848
Current Trial:  849
Current Trial:  850
Current Trial:  851
Current Trial:  852
Current Trial:  853
Current Trial:  854
Current Trial:  855
Current Trial:  856
Current Trial:  857
Current Trial:  858
Current Trial:  859
Current Trial:  860
Current Trial:  861
Current Trial:  862
Current Trial:  863
Current Trial:  864
Current Trial:  865
Current Trial:  866
Current Trial:  867
Current Trial:  868
Current Trial:  869
Current Trial:  870
Current Trial:  871
Current Trial:  872
Current Trial:  873
Current Trial:  874
Current Trial:  875


In [139]:
X.shape

(12480, 48, 1001)

In [140]:
X_red = X[:, :, :400]
X_red.shape

(12480, 48, 400)

In [141]:
X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])).shape

(12480, 19200)

In [143]:
len(y_event)

12480

In [144]:
X_train, X_eval, y_train, y_eval = train_test_split(
    X_red.reshape((X_red.shape[0], X_red.shape[1]*X_red.shape[2])), y_event, test_size=0.2, random_state=42
)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((9984, 19200), (2496, 19200), (9984,), (2496,))

In [138]:
X_eval

array([[-4.92295503616333, -4.274316787719727, -3.530621290206909, ...,
        -7.51123046875, -7.144469738006592, -6.726459980010986],
       [-3.99238920211792, -4.573351860046387, -5.19865083694458, ...,
        3.0889878273010254, 2.508065938949585, 1.9449099302291868],
       [-1.3690567016601562, -1.2272512912750242, -1.1418002843856812,
        ..., -22.24218368530273, -21.508632659912102, -20.72554016113281],
       ...,
       [-2.7587580680847164, -2.7292356491088867, -2.6308388710021973,
        ..., -3.3942432403564453, -3.685460805892944,
        -3.9943373203277583],
       [-18.59305763244629, -18.24982643127441, -17.805282592773438, ...,
        -1.7376010417938232, -2.151963472366333, -2.5701889991760254],
       [-2.782504796981811, -3.28445816040039, -3.9294309616088863, ...,
        4.226086616516113, 3.826579332351685, 3.411402463912964]],
      dtype=object)

In [145]:
clf = LazyClassifier(verbose=0,ignore_warnings=False, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

  3%|█▎                                     | 1/29 [41:12<19:13:54, 2472.68s/it]

AdaBoostClassifier model failed to execute
Found unknown categories [0.2525259852409363, -0.8598867654800415, 1.292681336402893, 3.183302402496338, -0.8613868355751038, 5.480263710021973, 5.754245281219482, 7.645411014556885, 2.5729074478149414, 9.05689811706543, 2.214402437210083, 11.633194923400875, 12.08674144744873, 10.165040016174316, 2.036757230758667, 6.0154571533203125, 16.53035545349121, 17.29422378540039, 17.365732192993164, 10.148972511291504, 2.824585437774658, 4.178518295288086, 4.077199459075928, 21.03557777404785, 4.676352500915527, 4.1976189613342285, 5.023715019226074, 5.073303699493408, 5.22401237487793, 25.74452400207519, 4.152841567993164, -0.5547786951065063, 6.72511100769043, 32.0978889465332, 32.04684066772461, 6.314841270446777, 7.587088108062744, 7.873989582061768, 6.133274078369141, 7.187895774841309, 1.6785540580749512, 8.332972526550293, 8.735750198364258, 8.033559799194336, 7.766026496887207, 7.471929550170898, 9.634332656860352, 9.221650123596191, 9.134626

  7%|██▌                                  | 2/29 [1:28:28<20:08:44, 2686.10s/it]

BaggingClassifier model failed to execute
Found unknown categories [0.2525259852409363, -0.8598867654800415, 1.292681336402893, 3.183302402496338, -0.8613868355751038, 5.480263710021973, 5.754245281219482, 7.645411014556885, 2.5729074478149414, 9.05689811706543, 2.214402437210083, 11.633194923400875, 12.08674144744873, 10.165040016174316, 2.036757230758667, 6.0154571533203125, 16.53035545349121, 17.29422378540039, 17.365732192993164, 10.148972511291504, 2.824585437774658, 4.178518295288086, 4.077199459075928, 21.03557777404785, 4.676352500915527, 4.1976189613342285, 5.023715019226074, 5.073303699493408, 5.22401237487793, 25.74452400207519, 4.152841567993164, -0.5547786951065063, 6.72511100769043, 32.0978889465332, 32.04684066772461, 6.314841270446777, 7.587088108062744, 7.873989582061768, 6.133274078369141, 7.187895774841309, 1.6785540580749512, 8.332972526550293, 8.735750198364258, 8.033559799194336, 7.766026496887207, 7.471929550170898, 9.634332656860352, 9.221650123596191, 9.1346263

 10%|███▊                                 | 3/29 [1:48:44<14:33:05, 2014.82s/it]

...

 97%|████████████████████████████████████▋ | 28/29 [11:05:12<17:21, 1041.23s/it]

XGBClassifier model failed to execute
'bool' object has no attribute 'all'


100%|██████████████████████████████████████| 29/29 [11:28:23<00:00, 1424.27s/it]

LGBMClassifier model failed to execute
Found unknown categories [0.2525259852409363, -0.8598867654800415, 1.292681336402893, 3.183302402496338, -0.8613868355751038, 5.480263710021973, 5.754245281219482, 7.645411014556885, 2.5729074478149414, 9.05689811706543, 2.214402437210083, 11.633194923400875, 12.08674144744873, 10.165040016174316, 2.036757230758667, 6.0154571533203125, 16.53035545349121, 17.29422378540039, 17.365732192993164, 10.148972511291504, 2.824585437774658, 4.178518295288086, 4.077199459075928, 21.03557777404785, 4.676352500915527, 4.1976189613342285, 5.023715019226074, 5.073303699493408, 5.22401237487793, 25.74452400207519, 4.152841567993164, -0.5547786951065063, 6.72511100769043, 32.0978889465332, 32.04684066772461, 6.314841270446777, 7.587088108062744, 7.873989582061768, 6.133274078369141, 7.187895774841309, 1.6785540580749512, 8.332972526550293, 8.735750198364258, 8.033559799194336, 7.766026496887207, 7.471929550170898, 9.634332656860352, 9.221650123596191, 9.1346263885

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


In [133]:
# models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/time-open-close-vs-rest-400ms.csv")